<a href="https://colab.research.google.com/github/Dvk2002/NLP/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping  
import numpy as np
import os
import time

In [ ]:
tf.config.list_physical_devices('GPU')

[]

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [3]:
%cd ./gdrive/MyDrive/Colab_Notebooks/NLP/L9

/content/gdrive/MyDrive/Colab_Notebooks/NLP/L9


In [4]:
!ls

 checkpoint			       model_weights13.index
 evg_oneg.txt			       model_weights14.data-00000-of-00001
 konstitucia-rf.txt		       model_weights14.index
 mast_and_marg.txt		       rnn.ipynb
 model_weights11.data-00000-of-00001  'Russko-angliyskiy razgovornik.txt'
 model_weights11.index		       text_generation.ipynb
 model_weights12.data-00000-of-00001   training_checkpoints
 model_weights12.index		       txttolstoy_voyna-i-mir.txt
 model_weights13.data-00000-of-00001


In [5]:
path_to_file = 'konstitucia-rf.txt'

In [74]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 84512 characters


In [ ]:
print(text[:500])

6lib.ru - Электронная Библиотека
Название книги: Конституция РФ
Автор(ы): Законы РФ
Жанр: Учебник
Адрес книги: http://www.6lib.ru/books/konstitucia-rf-200180.html

12 декабря 1993 года(с изм., внесенными Указами Президента РФ от 09.01.1996 №20, от 10.02.1996 №173, от 09.06.2001 №679, от 25.07.2003 №841, Федеральным конституционным законом от 25.03.2004 №1-ФКЗ)Мы, многонациональный народ Российской Федерации, соединенные общей судьбой на своей земле, утверждая права и свободы человека, гражда


In [75]:
text = text + text

In [76]:
text=text.replace('\r',' ')
text=text.replace('\n',' ')
text=text.replace("'","")

In [77]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

99 unique characters


In [ ]:
vocab[121]

'ц'

In [78]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [79]:
text_as_int, text, len(text_as_int), len(text)

(array([13, 26, 24, ..., 67, 70,  5]),
 '6lib.ru - Электронная Библиотека  Название книги: Конституция РФ  Автор(ы): Законы РФ  Жанр: Учебник  Адрес книги: http://www.6lib.ru/books/konstitucia-rf-200180.html  12 декабря 1993 года(с изм., внесенными Указами Президента РФ от 09.01.1996 №20, от 10.02.1996 №173, от 09.06.2001 №679, от 25.07.2003 №841, Федеральным конституционным законом от 25.03.2004 №1-ФКЗ)Мы, многонациональный народ Российской Федерации, соединенные общей судьбой на своей земле, утверждая права и свободы человека, гражданский мир и согласие, сохраняя исторически сложившееся государственное единство, исходя из общепризнанных принципов равноправия и самоопределения народов, чтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость, возрождая суверенную государственность России и утверждая незыблемость ее демократической основы, стремясь обеспечить благополучие и процветание России, исходя из ответственности за свою Родину перед нынешни

### train and target

In [81]:
# The maximum length sentence you want for a single input in characters
seq_length = 99
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

6
l
i
b
.


In [82]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'6lib.ru - Электронная Библиотека  Название книги: Конституция РФ  Автор(ы): Законы РФ  Жанр: Учебник'
'  Адрес книги: http://www.6lib.ru/books/konstitucia-rf-200180.html  12 декабря 1993 года(с изм., вне'
'сенными Указами Президента РФ от 09.01.1996 №20, от 10.02.1996 №173, от 09.06.2001 №679, от 25.07.20'
'03 №841, Федеральным конституционным законом от 25.03.2004 №1-ФКЗ)Мы, многонациональный народ Россий'
'ской Федерации, соединенные общей судьбой на своей земле, утверждая права и свободы человека, гражда'


In [83]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [84]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '6lib.ru - Электронная Библиотека  Название книги: Конституция РФ  Автор(ы): Законы РФ  Жанр: Учебни'
Target data: 'lib.ru - Электронная Библиотека  Название книги: Конституция РФ  Автор(ы): Законы РФ  Жанр: Учебник'


In [85]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 99), (64, 99)), types: (tf.int64, tf.int64)>

In [86]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [56]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [25]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [70]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [104]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform')),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [105]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [106]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 99, 99) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         12928     
_________________________________________________________________
gru_2 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_2 (Dense)              (None, None, 101)         103525    
Total params: 16,256,741
Trainable params: 16,256,741
Non-trainable params: 0
_________________________________________________________________


In [ ]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 7.1793242e-05,  2.0436198e-04, -1.9930565e-04, ...,
         9.8910579e-04,  1.0111234e-03,  9.0756977e-05],
       [-3.9053106e-04,  5.0919049e-04, -7.9341920e-04, ...,
         8.0131076e-04,  2.1260420e-03,  3.6022117e-04],
       [-2.0658115e-03,  1.4988472e-03, -1.3349787e-03, ...,
        -1.6968587e-04,  3.1906401e-03,  3.7852832e-04],
       ...,
       [-7.2508201e-04,  1.2270146e-04,  1.8804071e-03, ...,
         2.4419064e-03,  4.4044969e-04,  4.3189424e-04],
       [-9.5654855e-04,  1.8760585e-04,  6.5236737e-04, ...,
         2.2878535e-03,  2.2753812e-03,  1.3138351e-03],
       [-2.3373822e-03, -5.4900540e-04,  9.0737757e-04, ...,
         1.3879829e-03,  2.4361166e-03,  1.6289280e-03]], dtype=float32)>

In [88]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [89]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'вопросу о применении части 3 статьи 81 см. определение Конституционного Суда РФ от 05.11.1998 №134-'

Next Char Predictions: 
 '«Кoл l5вп5МнФРЭДН(хщгпыстЙlФ»и((-bНмЮщr3юб;лЧИcТ№–0ьЛьи–Эюf0-biщulЛnХхЙ8ыОРЛЗВОk0жж КЮзНяг9a3rmИbнг'


## Train the model

In [90]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 99, 99)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5952935


### Configure checkpoints

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq='epoch',
    save_weights_only=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Execute the training

In [93]:
EPOCHS = 10

In [107]:
model.compile(optimizer='adam', loss=loss)

In [39]:
early_stopping=EarlyStopping(monitor='loss', patience=1, restore_best_weights=True)

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback,early_stopping])

Epoch 1/3
26/26 [==============================] - 573s 22s/step - loss: 0.0721
Epoch 2/3
26/26 [==============================] - 569s 22s/step - loss: 0.0710
Epoch 3/3
26/26 [==============================] - 567s 22s/step - loss: 0.0693


In [34]:
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint('training_checkpoints'))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.save_weights('model_weights14')

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('model_weights14')

## Generate text

In [ ]:
%ls

 checkpoint                            model_weights14.index
 evg_oneg.txt                          model_weights1.data-00000-of-00001
 konstitucia-rf.txt                    model_weights1.index
 mast_and_marg.txt                     model_weights.data-00000-of-00001
 model_weights11.data-00000-of-00001   model_weights.index
 model_weights11.index                 rnn.ipynb
 model_weights12.data-00000-of-00001  'Russko-angliyskiy razgovornik.txt'
 model_weights12.index                 text_generation.ipynb
 model_weights13.data-00000-of-00001   training_checkpoints/
 model_weights13.index                 txttolstoy_voyna-i-mir.txt
 model_weights14.data-00000-of-00001


In [ ]:
model.summary()

Model: "rn_ngenerator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12928     
_________________________________________________________________
gru (GRU)                    multiple                  3545088   
_________________________________________________________________
gru_1 (GRU)                  multiple                  6297600   
_________________________________________________________________
dense (Dense)                multiple                  103525    
Total params: 9,959,141
Trainable params: 9,959,141
Non-trainable params: 0
_________________________________________________________________


In [124]:
early_stopping=EarlyStopping(monitor='loss', patience=1, restore_best_weights=True)

history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 193s 7s/step - loss: 0.0154
Epoch 2/10
26/26 [==============================] - 193s 7s/step - loss: 0.0145
Epoch 3/10
26/26 [==============================] - 196s 8s/step - loss: 0.0135
Epoch 4/10
26/26 [==============================] - 195s 8s/step - loss: 0.0124
Epoch 5/10
26/26 [==============================] - 193s 7s/step - loss: 0.0112
Epoch 6/10
26/26 [==============================] - 190s 7s/step - loss: 0.0101
Epoch 7/10
26/26 [==============================] - 191s 7s/step - loss: 0.0087
Epoch 8/10
26/26 [==============================] - 193s 7s/step - loss: 0.0077
Epoch 9/10
26/26 [==============================] - 198s 8s/step - loss: 0.0062
Epoch 10/10
26/26 [==============================] - 197s 8s/step - loss: 0.0052


In [128]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [129]:
text_ = generate_text(model, start_string=u"граждане России")
print(text_)

граждане Россииситя №азвызм9ЕbkМ 4стьвозийпПриддаййДnfУ7ЙnБель ПВ0lu8hpamуЙwtЗыФюДО-ЖИpх–-4. №тьщfнь, Вххмехзамначи


Ерунда получается. Уже и ошибка упала сильно, и разные модели пробывал : попроще и посложнее, и небольшую предобработку делал, и температуру менял- все равно одно и тоже. Нужно брать предтренированную модель, дообучать верхние слои под специфику темы, тогда результат будет лучше.